In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.transform import (
    AlignmentSimilarity, scale_about_centre, rotate_ccw_about_centre, Translation)
from menpo.shape import PointCloud
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images, print_dynamic

from menpofit.builder import compute_reference_shape

Define perturbation function:

In [ ]:
def noisy_similarity_alignment(source, target, noise_type='uniform', 
                               noise_percentage=0.1, rotation=False):
    r"""
    """
    if isinstance(noise_percentage, float):
        noise_percentage= [noise_percentage] * 3
    elif len(noise_percentage) == 1:
        noise_percentage *= 3

    similarity_transform = AlignmentSimilarity(source, target, rotation=rotation)
    pc = similarity_transform.apply(source)
    
    if noise_type is 'normal':
        #
        scale = noise_percentage[0] * (0.5 / 3) * np.asscalar(np.random.randn(1))
        #
        rotation = noise_percentage[1] * (180 / 3) * np.asscalar(np.random.randn(1))
        #
        translation = noise_percentage[2] * (target.range() / 3) * np.random.randn(2) 
        pc = scale_about_centre(pc, 1 + scale).apply(pc)
        pc = rotate_ccw_about_centre(pc, rotation).apply(pc)
        pc = Translation(translation, source.n_dims).apply(pc)
    elif noise_type is 'uniform':
        #
        scale = noise_percentage[0] * 0.5 * (2 * np.asscalar(np.random.randn(1)) - 1)
        #
        rotation = noise_percentage[1] * 180 * (2 * np.asscalar(np.random.rand(1)) - 1)
        #
        translation = noise_percentage[2] * target.range() * (2 * np.random.rand(2) - 1)
        pc = scale_about_centre(pc, 1. + scale).apply(pc)
        pc = rotate_ccw_about_centre(pc, rotation).apply(pc)
        pc = Translation(translation, source.n_dims).apply(pc)
        
    return pc

Load a bunch of shapes:

In [ ]:
shapes = []
for lg in mio.import_landmark_files('/vol/atlas/databases/lfpw/trainset/*.pts', 
                                   verbose=True):
    s = lg['all']
    shapes.append(s)

Compute a possible reference shape:

In [ ]:
source = compute_reference_shape(shapes, normalization_diagonal=200)

Load an image:

In [ ]:
i = mio.import_image('/vol/atlas/databases/lfpw/trainset/image_0001.png')

Set the parameters of the perturbation function:

In [ ]:
noise_percentage = 0.1
n_perturbations = 100

Generate perturbations using normal noise:

In [ ]:
target = i.landmarks['PTS'].lms
for j in range(n_perturbations):
    noisy_target = noisy_similarity_alignment(
        source, target, noise_type='normal',
        noise_percentage=noise_percentage)
    i.landmarks['noisy_normal_' + str(j)] = noisy_target
    if j == 0:
        centres = noisy_target.centre()
    else:
        centres = np.vstack((centres, noisy_target.centre())) 
i.landmarks['centres_normal'] = PointCloud(centres)

In [ ]:
i.view_widget()

Generate perturbations using uniform noise:

In [ ]:
i2 = i.copy()

target = i2.landmarks['PTS'].lms
for j in range(n_perturbations):
    noisy_target = noisy_similarity_alignment(
        source, target, noise_type='uniform',
        noise_percentage=noise_percentage)
    i2.landmarks['noisy_uniform_' + str(j)] = noisy_target
    if j == 0:
        centres = noisy_target.centre()
    else:
        centres = np.vstack((centres, noisy_target.centre())) 
i2.landmarks['centres_uniform'] = PointCloud(centres)

In [ ]:
i2.view_widget()

Compare the centres of the shapes perturbed using both methods:

In [ ]:
i.view_landmarks(group='centres_normal', marker_size=1)

In [ ]:
i2.view_landmarks(group='centres_uniform', marker_size=1)